In [19]:
from keras.datasets import mnist
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from Module import *
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train=torch.from_numpy(x_train).reshape((-1,784)).float()
x_test=torch.from_numpy(x_test).reshape((-1,784)).float()
y_train=torch.from_numpy(y_train).long()
y_test=torch.from_numpy(y_test).long()
scaler = preprocessing.Normalizer().fit(x_train)
x_train = torch.tensor(scaler.transform(x_train))
x_test= torch.tensor(scaler.transform(x_test))

In [4]:
def get_classes(size_thresh,entries,x_train,y_train,dim):
    model1=HDCModel(1000,784,dim,classh=True)
    classifier=FebHDVersion2(64,.0001)
    classifier=TrainRepeater(classifier,5)
    train_start(model1,classifier,x_train,y_train)
    yhat1=model1(x_train)
    print("done with initial training")
    accs=torch.zeros(len(yhat1.unique()))
    sizes=torch.zeros(len(yhat1.unique()))
    classification=torch.zeros(len(yhat1.unique()))
    for i,ekans in enumerate(yhat1.unique()):
        # arbok=yhat2[2]y_train==yhat
        # print(ekans)
        # print(arbok)
        # print(y_train[yhat1==ekans].mode().values)
        # print(y_train[yhat1==ekans][:10])
        # print((y_train[yhat1==ekans]==y_train[yhat1==ekans].mode().values).float().mean())
        classification[i]=y_train[yhat1==ekans].mode().values
        istrue=(y_train[yhat1==ekans]==y_train[yhat1==ekans].mode().values).float()
        accs[i]=istrue.mean()
        sizes[i]=len(istrue)
    print("done with computing metrics")
    hypervectors=torch.zeros(len(yhat1.unique()),model1.dimensionality)
    for i,ekans in enumerate(yhat1.unique()):
        hypervectors[i]=model1.encode(x_train[yhat1==ekans]).sum(0)
    print("done with computing centroids")
    avgsims=torch.zeros(len(yhat1.unique()))
    for j,i in enumerate(yhat1.unique()):
        ekans=yhat1[i]
        # arbok=yhat2[2]
        # print(ekans)
        # print(arbok)
        avgsims[j]=cos_cdist(model1.encode(x_train[yhat1==ekans]),hypervectors[j].unsqueeze(0)).mean()
    print("done with computing cosine deviation")
    is_optimized=sizes>size_thresh
    entries=len(avgsims[is_optimized])
    order=-avgsims[is_optimized].argsort()
    print(sum(accs[is_optimized][order]*sizes[is_optimized][order])/sum(sizes[is_optimized][order]))
    yhat1.unique()[is_optimized][order][:entries]
    grouping=[]
    for i in range(0,entries):
        grouping.append((yhat1==yhat1.unique()[is_optimized][order][i]).nonzero().squeeze().tolist())
    print("done with computing groupings")
    return grouping

In [5]:
def uniformity_report(index_list):
    index_list=list(set(index_list))
    realvals=torch.zeros(len(index_list))
    for i, index in enumerate(index_list):
        realvals[i]=y_train[index]
    mode=realvals.mode()
    return mode.values, len(y_train[index_list][y_train[index_list]==mode.values])/len(index_list),len(index_list)

In [6]:
def combine_groupings(comparison_grouping, category):
    new_indices=[]
    for i in range(0,len(list(category))):
        new_indices+=comparison_grouping[list(category)[i]]
    return new_indices

In [7]:
import matplotlib.pyplot as plt
import networkx as nx

def get_graph_from_adjacency_matrix(adjacency_matrix):
    rows, cols = np.where(adjacency_matrix == 1)
    edges = zip(rows.tolist(), cols.tolist())
    gr = nx.Graph()
    gr.add_edges_from(edges)
    return gr

In [ ]:
#Step 1: Generate many homogenous clusters (febHD): 

In [8]:
dim=2000

size_thresh=25
entries=10

# sum(accs[is_optimized]*sizes[is_optimized])/sum(sizes[is_optimized])
# original_grouping=get_classes(size_thresh,entries,x_train,y_train,dim)


# generate overlapping groupings (may not even contain all of the points)

comparison_grouping=[]
for i in range(0,3):
    comparison_grouping+=get_classes(size_thresh,entries,x_train,y_train,dim)

#turn points into similarity matrix
similarity_matrix=torch.zeros(len(comparison_grouping),len(comparison_grouping))
for i,list_ in enumerate(comparison_grouping):
    for j,comparison_list in enumerate(comparison_grouping):
        similarity_matrix[i][j]=len(set(list_).intersection(comparison_list))

#establish overlap threshold for generating adjacency matrix
adjacency_matrix=similarity_matrix>=10
#turn adjacency matrix into graph in preparation for community detection algorithm
gr=get_graph_from_adjacency_matrix(adjacency_matrix)
connected_components=nx.connected_components(gr)
subgraphs=[gr.subgraph(c) for c in connected_components]


done with initial training
done with computing metrics
done with computing centroids
done with computing cosine deviation
tensor(0.9104)
done with computing groupings
done with initial training
done with computing metrics
done with computing centroids
done with computing cosine deviation
tensor(0.9136)
done with computing groupings
done with initial training
done with computing metrics
done with computing centroids
done with computing cosine deviation
tensor(0.9207)
done with computing groupings


In [ ]:
#Step 2: Detect more than 10 communities using community detection

In [9]:
#Louvain community detection algorithm (Newman Eigenvector algorithm may also be appropriate)
categories=nx.community.louvain_communities(gr, threshold=1e-10, seed=None)
print(len(categories))
sum_=0
denom=0
clusters=[]
for i in range(0,len(categories)):
    cluster=combine_groupings(comparison_grouping,categories[i])
    denom+=len(cluster)*uniformity_report(cluster)[1]
    sum_+=len(cluster)
    print(uniformity_report(cluster))
    print(len(cluster))
    clusters.append(cluster)
print(denom/sum_)

14
(tensor(8.), 0.7786432516640702, 7061)
16979
(tensor(3.), 0.815024805102764, 7055)
17216
(tensor(0.), 0.9152037617554859, 6380)
17900
(tensor(4.), 0.832271396590322, 5807)
13548
(tensor(9.), 0.6951713969775156, 5426)
12371
(tensor(7.), 0.8996827315304426, 6619)
17556
(tensor(2.), 0.8070996147495872, 7268)
18758
(tensor(9.), 0.5836538461538462, 4160)
9428
(tensor(5.), 0.6837385756164857, 5799)
12748
(tensor(6.), 0.8969641214351426, 6522)
17850
(tensor(2.), 0.6491228070175439, 57)
57
(tensor(5.), 0.9003099173553719, 1936)
4499
(tensor(1.), 0.9617984131648546, 6806)
19805
(tensor(7.), 0.37142857142857144, 35)
35
0.8269812043263712


In [10]:
#just old stuff for reference
targets=torch.zeros(len(y_train)) #new yhat for new classes to emulate umap
i=0
for indices in clusters:
    for index in indices:
        targets[index]==i
    i+=1
targets=targets.long()

In [ ]:
cos_cdist(model.encode(x), model.classes)

NameError: name 'self' is not defined

In [11]:
#step 3: resolve overlapping class assignments for each point
for i in range(len(clusters)):
    clusters[i]=torch.tensor(clusters[i])
category_membership=torch.zeros(len(x_train))
for index in range(len(x_train)):
    assignment_vector=torch.zeros(len(clusters))
    for i in range(len(clusters)):
        assignment_vector[i]=len(clusters[i][clusters[i]==index])

    category_membership[index]=assignment_vector.argmax()

In [12]:
#Step 4: train an HDC model on the most common 10 classes
model=HDCModel(classes=len(clusters),features=784,dim=2000,classh=True)
classifier=ClassicVersion1(64,1)
# classifier=ClassicVersion0(64,1)
train_start(model,classifier,x_train,category_membership.long())

In [13]:
eval_acc(model,x_train,category_membership)

tensor(0.9344)

In [61]:
yhat=model(x_train)

In [67]:
lengths=torch.zeros(len(model.classes))

for i in range(0,len(model.classes)):
    lengths[i]=len(yhat[yhat==i])
    

In [74]:
model.classes=model.classes[torch.topk(lengths,10).indices] #only assign the top 10 classes

In [75]:
yhat=model(x_train)

In [78]:
#Step 5: compute accuracy
eval_inferred_acc(model,x_train,y_train)

tensor(0.8725)

In [ ]:
#end

In [ ]:
#extra stuff

In [14]:
category_membership.unique()

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 11., 12.])

In [52]:
len(category_membership[category_membership==12])

6562

In [18]:
from sklearn.metrics import fowlkes_mallows_score
fowlkes_mallows_score(category_membership,y_train)

0.8192413342522029

In [15]:
labels=category_membership
yhat=torch.zeros(y_train.shape)
modes=[]
for i in torch.unique(y_train):
    n=torch.mode(y_train[labels==i]).values
    modes.append(n)
for i in range(y_train.shape[0]):
    yhat[i]=modes[int(labels[i])]
eval=[yhat[i]==y_train[i] for i in range(len(y_train))]
print(sum(eval)/len(y_train))

IndexError: list index out of range

In [ ]:
for i in range(0,len(clusters)):
    print(len(labels[labels==i]))

6595
5152
5964
5897
6967
5950
5319
5894
6207
6055


In [ ]:
category_membership.shape

torch.Size([60000])

In [31]:
similarities=cos_cdist(model.encode(x_train), model.classes)

In [21]:
print(similarities.shape)

torch.Size([60000, 14])


In [36]:
similarities=similarities.numpy()

In [37]:
reduced_dim_vec = similarities
vectors_list = np.array_split(reduced_dim_vec, len(reduced_dim_vec))
category_assignments=pd.Series(vectors_list).apply(lambda x: x.squeeze())

In [38]:
category_assignments

0        [0.57468, 0.6285479, 0.5455284, 0.5325451, 0.5...
1        [0.5372889, 0.5457319, 0.650884, 0.539474, 0.5...
2        [0.5048593, 0.5196212, 0.4901391, 0.5671302, 0...
3        [0.5663998, 0.54452395, 0.51290345, 0.5775483,...
4        [0.58430296, 0.5444071, 0.5390455, 0.61799, 0....
                               ...                        
59995    [0.6090697, 0.56485844, 0.52651197, 0.5605037,...
59996    [0.6055812, 0.6474652, 0.53547484, 0.5355785, ...
59997    [0.55851996, 0.57203615, 0.54585, 0.54019326, ...
59998    [0.5564495, 0.5572184, 0.56590635, 0.5607771, ...
59999    [0.58963406, 0.5420234, 0.55819845, 0.57017237...
Length: 60000, dtype: object

In [79]:
category_membership.long().dtype

torch.int64